## Modelo LDA de rotulagem de dados

Esse notebook é uma implementação do modelo de classificação LDA

### Install

In [1]:
!pip install texthero nltk -q

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install pyLDAvis

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Imports

In [35]:
import pandas as pd
import numpy as np


from collections import Counter
import nltk
from nltk.stem.porter import *
nltk.download('stopwords')

import texthero as hero
from texthero import preprocessing
import plotly.express as px

import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import matplotlib.pyplot as plt

from nltk.corpus import stopwords

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guilherme.gomes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Leitura dos dados

In [4]:
df = pd.read_csv("data_full_lion_bbq.csv")

In [8]:
select = ['video_title', 'comment', 'author',
       'comment_cleaned', 'comment_stop_word', 'comment_tokenized',
       'comment_stemming', 'comment_lematized']

In [11]:
df = df[[*select]]

### pré-processamento

In [21]:
tw = hero.visualization.top_words(df['comment_stop_word']).head(15)
tw
# fig = px.bar(tw)
# fig.show()
# tw.head()

leo         493
top         433
parabens    336
manolo      272
nao         263
video       250
bom         250
pra         248
vc          214
carne       203
ai          194
abraco      168
cara        165
so          152
demais      146
Name: comment_stop_word, dtype: int64

In [20]:
tw = hero.visualization.top_words(df['comment_stop_word']).tail(10)
tw

pronto                 1
poderias               1
karamba                1
tubarao                1
mantenhase             1
inovacoes              1
beijo                  1
sensacionaaaallllll    1
vegetais;hortalicas    1
sinceros               1
Name: comment_stop_word, dtype: int64

In [39]:
# stop_words = ['leo','top','parabens', 'manolo','video','bom','pra','vc','carne','ai',
#               'abraco','cara','so','demais']
# df['comment_stop_word'] = df['comment_stop_word'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

### feature engineering

In [40]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if len(token) > 3:
            result.append(token)
    return result

In [41]:
processed_docs = df['comment_stop_word'].map(preprocess)
processed_docs[:10]

0    [mini, junta, pouco, gente, leva, nome, casame...
1    [deus, abencoe, sempre, familia, parabens, bel...
2       [quale, manollo, abre, pros, entrar, parabens]
3     [manolo, voce, arrebentou, esqueceu, hidratacao]
4    [caramba, amigos, orgulho, maximo, festival, p...
5    [excelente, ambiente, descontraido, leve, tudo...
6    [familia, amizades, legais, manolo, parabens, ...
7    [demais, maravilha, senhor, nome, almiro, toda...
8    [manolo, show, parabens, campinas, tomei, aqui...
9    [manolo, emmmm, parabens, mini, evento, famili...
Name: comment_stop_word, dtype: object

In [56]:
id2word = gensim.corpora.Dictionary(processed_docs)

# less than 15 documents (absolute number) or
# more than 0.5 (50%) documents (fraction of total corpus size, not absolute number).
# after the above two steps, keep only the first 100000 most frequent tokens.

id2word.filter_extremes(no_below=5, no_above=0.5, keep_n=100)

# Term Document Frequency
corpus = [id2word.doc2bow(doc) for doc in processed_docs]

In [57]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('gente', 1), ('kkkkkk', 1)]]